In [1]:
import time

import matplotlib.image as mpimg
import matplotlib.pyplot as plt
%matplotlib inline
%load_ext autoreload
%autoreload 2
import numpy as np

import os, sys

from preprocessing import *
import image_preprocessing

from skimage import color

import keras

from skimage.color import rgb2hsv

from keras.layers import Conv2D, Dense, Flatten, MaxPooling2D, LeakyReLU, Dropout

Using TensorFlow backend.


In [2]:
ROOT_DIR = 'training/'
IMAGE_DIR = ROOT_DIR + 'images/'
GT_DIR = ROOT_DIR + 'groundtruth/'

PATCH_SIZE = 10

WINDOW_SIZE = 71

In [ ]:
files = os.listdir(IMAGE_DIR)

imgs = np.stack([load_image(IMAGE_DIR + file) for file in files]) # images (400, 400, 3)
gt_imgs = np.stack([load_image(GT_DIR + file) for file in files]) # images (400, 400)

patched_imgs = np.stack([patch_image(img, PATCH_SIZE) for img in imgs]) # images (400, 400)
patched_gts = np.stack([patch_groundtruth(gt, PATCH_SIZE) for gt in gt_imgs])

In [ ]:
PATCHED_SIZE = imgs.shape[1] // PATCH_SIZE
WINDOWS_PER_IMAGE = PATCHED_SIZE ** 2

In [ ]:
N = 1 # Number of image to be used in training

leakyness = 0.1

In [ ]:
windows_per_image = [image_to_neighborhoods(im, WINDOW_SIZE, True) for im in patched_imgs[:N]]
windows = np.vstack(windows_per_image)

window_labels = np.ravel(patched_gts[:N])
assert window_labels.shape[0] == windows.shape[0]

In [ ]:
window_labels.shape

In [ ]:
window_labels = keras.utils.np_utils.to_categorical(window_labels)

In [ ]:
window_cnn = keras.models.Sequential([
    
    Conv2D(32, (5, 5), strides=(1, 1), input_shape=windows.shape[1:]),
    LeakyReLU(leakyness),
    
    MaxPooling2D(2),
    Dropout(0.25),

    Conv2D(64, (3, 3), strides=(1, 1)),
    LeakyReLU(leakyness),
    
    MaxPooling2D(2),
    Dropout(0.25),
    
    Conv2D(128, (3, 3), strides=(1, 1)),
    LeakyReLU(leakyness),
    
    MaxPooling2D(2),
    Dropout(0.25),
    
    Conv2D(256, (3, 3), strides=(1, 1)),
    LeakyReLU(leakyness),

    MaxPooling2D(2),
    Dropout(0.25),
    
    Dense(128),
    LeakyReLU(leakyness),
    
    Flatten(),
    Dense(2, activation='sigmoid'),
])

In [ ]:
#window_cnn.summary()

In [ ]:
window_cnn.compile(loss=keras.losses.categorical_crossentropy, optimizer=keras.optimizers.Adam(lr=0.001), metrics=['accuracy'])

In [ ]:
nw = windows.shape[0]
def batch_generator():
    i = 0
    while True:
        print('Called', i)
        i+=1
        indices = np.random.choice(range(nw), 200, replace=False)
        yield (windows[indices], window_labels[indices])

In [ ]:
def smart_generator(n_windows, n_epochs):
    ws_per_epoch = n_windows // n_epochs
    indices = np.random.choice(range(n_windows), n_epochs*ws_per_epoch, replace=False)
    indices_per_epoch = [indices[i:i+ws_per_epoch] for i in range(0, n_windows, ws_per_epoch)]
    for batch_idx in indices_per_epoch:
        print(len(batch_idx))
        ws = windows[batch_idx]
        labs = window_labels[batch_idx]
        yield (ws, labs)

In [ ]:
epochs = 10

In [ ]:
start = time.perf_counter()

window_cnn.fit(windows, window_labels, epochs=epochs)#, batch_size=1600)
#window_cnn.fit_generator(smart_generator(windows.shape[0], epochs), steps_per_epoch=windows.shape[0] // epochs, epochs=10)

time.perf_counter() - start

In [ ]:
#window_cnn.save('backup')

# Test on a never seen image

In [ ]:
i = 0

In [ ]:
plt.imshow(imgs[i])

In [ ]:
window_t = image_to_neighborhoods(patched_imgs[i], WINDOW_SIZE, True)

In [ ]:
preds = window_cnn.predict(window_t)

In [ ]:
plt.plot(np.sort(np.ravel(preds)))

In [ ]:
preds = (preds[:,1] > preds[:,0]) * 1

In [ ]:
f = preds
f = np.reshape(f, (PATCHED_SIZE, PATCHED_SIZE))

In [ ]:
plt.imshow(patched_gts[i], cmap='Greys_r')

In [ ]:
plt.imshow(f, cmap='Greys_r')

In [ ]:
plt.imshow(np.hstack([patched_gts[i], f]), cmap='Greys_r')

# Test data set

In [4]:
from keras.models import load_model
import scipy
window_cnn = load_model('trained_model_full.hdf5')


In [105]:
TEST_DIR = 'test_set_images/'
PRED_DIR = 'pred_images/'
test_files = os.listdir(TEST_DIR)
WINDOW_SIZE = 71

['test_13']


In [106]:
for file in test_files:
    print(file)
    filename = '{}{}/{}.png'.format(TEST_DIR, file, file)
    img = load_image(filename)
    img_m = reflect_padding(img, 1)
    img_m = patch_image(img, 10)
    windows = image_to_neighborhoods(img_m, WINDOW_SIZE, True)
    print('Read and windowed', file)
    preds = window_cnn.predict(windows)
    print('Generated predictions')
    preds = (preds[:,1] > preds[:,0]) * 1
    preds = np.reshape(preds, img_m.shape[:2])
    preds = unpatch(preds, 10)
    preds = preds[1:-1, 1:-1]
    assert preds.shape == img.shape[:2]
    scipy.misc.imsave(PRED_DIR + file + '.png', preds)
    print('Saved array')

test_13
Read and windowed test_13
Generated predictions
Saved array
test_14
Read and windowed test_14
Generated predictions
Saved array
test_22
Read and windowed test_22
Generated predictions
Saved array
test_25
Read and windowed test_25
Generated predictions
Saved array
test_49
Read and windowed test_49
Generated predictions
Saved array
test_40
Read and windowed test_40
Generated predictions
Saved array
test_47
Read and windowed test_47
Generated predictions
Saved array
test_24
Read and windowed test_24
Generated predictions
Saved array
test_23
Read and windowed test_23
Generated predictions
Saved array
test_15
Read and windowed test_15
Generated predictions
Saved array
test_12
Read and windowed test_12
Generated predictions
Saved array
test_46
Read and windowed test_46
Generated predictions
Saved array
test_41
Read and windowed test_41
Generated predictions
Saved array
test_48
Read and windowed test_48
Generated predictions
Saved array
test_30
Read and windowed test_30
Generated pred

In [87]:
from mask_to_submission import *

In [108]:
pre_files = os.listdir(PRED_DIR)

In [114]:
image_filenames = []
for name in pre_files:
    image_filename = PRED_DIR + name
    image_filenames.append(image_filename)

In [115]:
masks_to_submission('sub_test.csv', *image_filenames)

pred_images/test_14.png
pred_images/test_28.png
pred_images/test_29.png
pred_images/test_15.png
pred_images/test_9.png
pred_images/test_17.png
pred_images/test_16.png
pred_images/test_8.png
pred_images/test_12.png
pred_images/test_13.png
pred_images/test_39.png
pred_images/test_11.png
pred_images/test_10.png
pred_images/test_38.png
pred_images/test_48.png
pred_images/test_49.png
pred_images/test_42.png
pred_images/test_43.png
pred_images/test_41.png
pred_images/test_40.png
pred_images/test_44.png
pred_images/test_50.png
pred_images/test_45.png
pred_images/test_47.png
pred_images/test_46.png
pred_images/test_3.png
pred_images/test_35.png
pred_images/test_21.png
pred_images/test_20.png
pred_images/test_34.png
pred_images/test_2.png
pred_images/test_22.png
pred_images/test_36.png
pred_images/test_37.png
pred_images/test_23.png
pred_images/test_1.png
pred_images/test_5.png
pred_images/test_27.png
pred_images/test_33.png
pred_images/test_32.png
pred_images/test_26.png
pred_images/test_4.png